In [2]:
from utils import authenticate

credentials, PROJECT_ID = authenticate()

In [ ]:
print(PROJECT_ID)

In [4]:
REGION = "us-central1"

In [5]:
import vertexai

In [6]:
vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

In [7]:
from google.cloud import bigquery

In [8]:
bq_client = bigquery.Client(project=PROJECT_ID, credentials=credentials)

In [9]:
QUERY_TABLES = """
SELECT
    table_name
FROM
    `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [10]:
query_job = bq_client.query(QUERY_TABLES)

In [11]:
for row in query_job:
    for value in row.values():
        print(value)

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


In [12]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [13]:
import pandas as pd

In [14]:
query_job = bq_client.query(INSPECT_QUERY)

In [15]:
stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
stack_overflow_df.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,64177830,How to read to a certain point in python for e...,<p>I tried to read from every line until the <...,NaN,2,0,NaT,2020-10-02 20:24:36.460000+00:00,NaN,2020-10-03 19:36:27.847000+00:00,2020-10-03 00:43:19.873000+00:00,None,7589636.0,user13798483,NaN,None,1,0,python|function|variables|line,256
1,64183547,"How to use nested ""map""","<p>I would like to map a json, based on some n...",64183941.0,2,0,NaT,2020-10-03 11:08:52.807000+00:00,NaN,2020-10-04 10:24:48.673000+00:00,NaT,None,NaN,None,11380853.0,None,1,2,dataweave|mulesoft,256
2,64220954,Azure DevOps Ephemeral agents not working,<p>I am trying to configure an Ephemeral agent...,64246902.0,2,0,NaT,2020-10-06 07:11:42.210000+00:00,1.0,2020-10-07 15:04:27.907000+00:00,NaT,None,NaN,None,5311841.0,None,1,0,azure|azure-devops|agent|azure-container-insta...,256


In [16]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
"""

In [17]:
query_job = bq_client.query(INSPECT_QUERY)

In [18]:
# will run into error

# stack_overflow_df = query_job\
#     .result()\
#     .to_arrow()\
#     .to_pandas()

# stack_overflow_df.head()

In [25]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body as output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
     `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [26]:
bq_job = bq_client.query(QUERY)

In [27]:
so_df = bq_job.result().to_arrow().to_pandas()
so_df.head(2)

,input_text,output_text
0,Writing nested list to file per line: Unsuppor...,<p>it seems like your code had a few bugs</p>\...
1,Whatsapp asking for updating chrome version<p>...,<p>The solution was to delete my saved chrome ...


In [28]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

In [43]:
so_df["input_text_instruction"] = INSTRUCTION_TEMPLATE + " " + so_df["input_text"]

In [46]:
so_df.head(2)

Index(['input_text', 'output_text', 'input_text_instruction'], dtype='object')

In [57]:
from sklearn.model_selection import train_test_split
### test_size=0.2 means 20% for evaluation and training set to be 80%
### random_state parameter is used to ensure random sampling for a fair comparison
train, evaluation = train_test_split(so_df, test_size=0.2, random_state=42)

In [ ]:
print(train)

In [52]:
import datetime
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")
print(date)

10:28:05:2024


In [60]:
cols = ['input_text_instruction', 'output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [61]:
training_data_filename = f"tune_data_stack_overflow_python_qa-{date}.jsonl"

In [62]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)